# Bias Uncovering Test Case (BTC) for Country-of-origin Bias

In [1]:
import os
import pandas as pd
import numpy as np
import math
import time
import pickle

from metric import evaluate_btc, evaluate_btc_mtnlp
from utils import accuracy
from utils import load_mutant_and_prediction

## Measuring the number of BTCs found by BiasFinder

In [2]:
mutation_tool = "biasfinder"
bias_type = "country"
task = "imdb"  # dataset used for fine-tuning
model = "bert-base-uncased"
mutant = "imdb"  # dataset used for generating mutants


df = load_mutant_and_prediction(mutation_tool, model, bias_type, mutant)

df.head()


,label,mutant,original,template,gender,mutation_type,template_id,prediction,original_prediction
0,1,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,Irish,act,58347,1,1
1,1,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,Israeli,act,58347,1,1
2,1,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,Ukrainian,act,58347,1,1
3,1,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,Canadian,act,58347,1,1
4,1,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,The Mascot is Ladislaw Starewicz 's masterpiec...,Norwegian,act,58347,1,1


In [3]:
print("Accuracy on mutants: {:.2f}%".format(accuracy(df["label"],df["prediction"])))


Accuracy on mutants: 91.36%


In [4]:
def print_evaluation(evaluation):
    print("# Mutants \t:", evaluation["mutant"])
    print("# Templates \t:", evaluation["template"])
    print("# BTCs \t\t:", evaluation["btc"])

evaluation = evaluate_btc(df["label"], df["prediction"], df["mutant"], df["template"], "gender", df["gender"])
print_evaluation(evaluation)

KeyboardInterrupt: 

## IMDB

Measure the number of bias uncovering test cases found by BiasFinder on models fine-tuned using IMDB dataset.

In [5]:
# models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
#           "albert-base-v2", "microsoft/mpnet-base", "facebook/muppet-roberta-base", "google/electra-base-generator"]

models = ["bert-base-uncased", "bert-base-cased", "roberta-base", 
          "albert-base-v2", "microsoft/mpnet-base", "facebook/muppet-roberta-base", "google/electra-base-generator"]


mutation_tools = ["biasfinder", "mtnlp"]
task = "imdb"  # dataset used for fine-tuning
mutant = "imdb"  # dataset used for generating mutants

d = pd.DataFrame(columns=["model", "tool",
                          "template", "mutant", "btc"])

for model in models:
    for mutation_tool in mutation_tools:
        df = load_mutant_and_prediction(
            mutation_tool, model, bias_type, mutant)
        if mutation_tool == "mtnlp" :
            evaluation = evaluate_btc_mtnlp(df)
        else : 
            evaluation = evaluate_btc(df["label"], df["prediction"],
                                  df["mutant"], df["template"], "gender", df["gender"])

        d = d.append(
            {
                "model": model,
                "tool": mutation_tool,
                "template": evaluation['template'],
                "mutant": evaluation['mutant'],
                "btc": evaluation['btc']
            },
            ignore_index=True)

d


,model,tool,template,mutant,btc
0,bert-base-uncased,biasfinder,2828,70700,3546
1,bert-base-cased,biasfinder,2828,70700,4794
2,roberta-base,biasfinder,2828,70700,2620
3,xlnet-base-cased,biasfinder,2828,70700,1174
4,albert-base-v2,biasfinder,2828,70700,3566
5,microsoft/mpnet-base,biasfinder,2828,70700,1692
6,facebook/muppet-roberta-base,biasfinder,2828,70700,2832
7,google/electra-base-generator,biasfinder,2828,70700,3004


## The total number of BTCs found

In [6]:
d.drop(columns=["model"]).groupby("tool").sum()

,template,mutant,btc
tool,,,
biasfinder,22624,565600,23228


## Twitter Sentiment 140

Measure the number of bias uncovering test cases found by BiasFinder models fine-tuned using Twitter Sentiment 140 dataset.

In [7]:
# models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
#           "albert-base-v2", "microsoft/mpnet-base", "facebook/muppet-roberta-base", "google/electra-base-generator"]

models = ["bert-base-uncased", "bert-base-cased", "roberta-base",
          "albert-base-v2", "microsoft/mpnet-base", "facebook/muppet-roberta-base", "google/electra-base-generator"]


mutation_tools = ["biasfinder", "mtnlp"]

task = "twitter_s140"  # dataset used for fine-tuning
mutant = "twitter_s140"  # dataset used for generating mutants

d = pd.DataFrame(columns=["model", "tool",
                          "template", "mutant", "btc"])

for model in models:
    for mutation_tool in mutation_tools:
        df = load_mutant_and_prediction(
            mutation_tool, model, bias_type, mutant)
        if mutation_tool == "mtnlp":
            evaluation = evaluate_btc_mtnlp(df)
        else:
            evaluation = evaluate_btc(df["label"], df["prediction"],
                                      df["mutant"], df["template"], "gender", df["gender"])

        d = d.append(
            {
                "model": model,
                "tool": mutation_tool,
                "template": evaluation['template'],
                "mutant": evaluation['mutant'],
                "btc": evaluation['btc']
            },
            ignore_index=True)

d


,model,tool,template,mutant,btc
0,bert-base-uncased,biasfinder,959,23975,4468
1,bert-base-cased,biasfinder,959,23975,4380
2,roberta-base,biasfinder,959,23975,6810
3,xlnet-base-cased,biasfinder,959,23975,5218
4,albert-base-v2,biasfinder,959,23975,4096
5,microsoft/mpnet-base,biasfinder,959,23975,3658
6,facebook/muppet-roberta-base,biasfinder,959,23975,5554
7,google/electra-base-generator,biasfinder,959,23975,4992


## The total number of BTCs found

In [8]:
d.drop(columns=["model"]).groupby("tool").sum()


,template,mutant,btc
tool,,,
biasfinder,7672,191800,39176
